## 整数规划问题的求解



这个系列的[第一篇文章](./IntegerOpt.ipynb)大概讲了Ortools的基本概况，安装和在线性规划问题上的使用场景。本部分涉及整数规划(Integer Programming)的部分，代码则涉及基础的IP问题求解，以及一种用数组形式代入求解器进行求解的方法。


- 混合整数规划：要求一些**决策变量是整数的线性优化问题**（Mixed Integer Programming, MIP）
    - 这些变量可能是一些无法切分的物品数量（电视机、汽车等）；
    - 也可以是0-1布尔变量，比如考虑一个人是否被分配到某个工作岗位；

- Google 提供了几种解决MIP问题的方法：
    - MPSolver：是多个第三方MIP求解器的包装，这些求解器使用标准的分支定界（Branch and Bound）技术。
    - CP-SAT 求解器：使用 SAT方法的约束规划求解器。
    - CP求解器：原始的Constraint Programming求解器。

- 使用场景：
    - MIP求解器更适合可以转化成具有任意整数变量的标准线性规划的问题；
    - CP-SAT更适合大多数变量为布尔值的问题。
    - 同样的，用于解决最小费用流问题的网络流求解器也是一个不错的解决方案。


-------


## 解决整数规划问题实战

给出如下整数规划问题：
$$\max x + 10y$$

$$\text{s.t.} \begin{align}\begin{equation*}
\begin{cases}
x + 7y \leq 17.5  \\
0 \leq x \leq 3.5  \\
0 \leq y  \\
\end{cases}
\end{equation*}\end{align} \\ 
x, y \in \text{Integers}
$$

我们使用MIP求解器来求解这个问题。默认的 OR-Tools 使用SCIP来求解 MIP 问题。代码思路和前一篇的类似，这里不重复，直接给出代码和注释（见图三及后续）

----------


对于一些参数较多的整数规划问题，我们可以使用数组形式代入求解器求解。

给出如下问题：

$$\max \hspace{5pt} 7x_1 + 8x_2 + 2x_3 + 9x_4 + 6x_5 \\ 
\text{s.t.} \begin{align}\begin{equation*}
\begin{cases}
5 x_1 + 7 x_2 + 9 x_3 + 2 x_4 +  x5  & \leq 250 \\
18 x_1 + 4 x_2 - 9 x_3 + 10 x_4 + 12 x_5  & \leq 285 \\
4 x_1 + 7x_2 + 3 x_3 + 8 x_4 + 5 x_5 &  \leq 211 \\
5 x_1 + 13 x_2 + 16 x_3 + 3 x_4 - 7 x_5  & \leq 315 \\
\end{cases}
\end{equation*}\end{align} \\ x_1, x_2 ... x_5 \in \text{non-negative integers}$$

这一部分的变量声明等都和前面的类似，不同的在于后面的代码使用`MakeRowConstraint` 方法（Python下，具体取决于编码语言）为示例创建约束。该方法的前两个参数是约束的下限和上限。第三个参数（约束的名称）是可选的。

对于每个约束，可以使用方法 `SetCoefficient` 定义变量的系数。该方法将约束 $i$ 中变量 $x[j]$ 的系数指定为数组 constraint_coeffs 的 $[i][j]$ 条目。

In [1]:
from ortools.linear_solver import pywraplp
def main():
    # 用SCIP后端创建MIP求解器
    solver = pywraplp.Solver.CreateSolver("SAT")
    if not solver:
        return

    infinity = solver.infinity()
    # X和Y都是非负整数变量
    x = solver.IntVar(0.0, infinity, "x")
    y = solver.IntVar(0.0, infinity, "y")

    print("Number of variables =", solver.NumVariables())
    # x + 7 * y <= 17.5.
    solver.Add(x + 7 * y <= 17.5)

    # x <= 3.5.
    solver.Add(x <= 3.5)

    print("Number of constraints =", solver.NumConstraints())

    # 目标函数
    solver.Maximize(x + 10 * y)

    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print("Objective value =", solver.Objective().Value())
        print("x =", x.solution_value())
        print("y =", y.solution_value())
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())


if __name__ == "__main__":
    main()

Number of variables = 2
Number of constraints = 2
Solving with CP-SAT solver v9.10.4067
Solution:
Objective value = 23.0
x = 3.0
y = 2.0

Advanced usage:
Problem solved in 15.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes


用列表形式描述整数规划问题

> 单个添加约束，步骤太复杂，可以用列表方式快速添加

> > PS 下面的第一个Python snippet是一个简单的模型求解。

In [2]:
from ortools.linear_solver import pywraplp

def solve_transportation_model():
    data = {}
    
    data["od_coefficient_matrix"] = [
        [15374.5,  21442.25 , 28408.5, 1000000,   1000000 ],
        [11279.8,  15212.35 , 20258.7, 1000000,   1000000 ],
        [10949.6,  14735.85 , 19708.9, 26466.75,  29847.7 ],
        [11066.5,  14924.25 , 20172.8, 14984.85,  33598.2 ],
        [10756.5,  14429.45 , 19286.2, 29261.85,  32231.1 ],
        [9325.7,   12143.65 , 16093.1, 22611.35,  25899.6 ]
    ]
    
    data["capacity_matrix"] = [
        [139.975323,215.902461,296.156393, 0,0,], 
        [73.309396,115.560991,162.250231, 0,0,], 
        [66.702968,105.941704,149.985791, 229.887984,263.421977], 
        [66.974115,110.912569,157.824413, 272.780953,316.081499], 
        [64.093506,102.381586,144.494524, 268.018274,261.147326], 
        [38.380769,61.520627,87.181579, 150.526594,180.921025], 
    ]
    
    data["vacant_cost"] = [
        500, 710, 820, 1170, 1360
    ]
    
    data["capacity_constraint"] = [
        1000, 800,700,3000,409, 1500
    ]
    
    data["total_ships"] = [
        40, 15, 8, 12, 10
    ]
    
    data["course"] = 6
    data["ships"] = 5
    
    # =========== Complete the hardcode data ===========
    
    solver = pywraplp.Solver.CreateSolver("SCIP")
    
    if not solver:
        return 
    
    x = {}
    infinity = solver.infinity()
    for i in range(data["course"]):
        for j in range(data["ships"]):
            x[i * data["ships"] + j] = solver.IntVar(0, infinity, f"x[{i}_{j}]" )
    print("Number of variables =", solver.NumVariables())
    
    for i in range(data["course"]):
        constraint = solver.RowConstraint(data["capacity_constraint"][i], infinity, f"constr_{i}")
        
        for j in range(data["ships"]):
            constraint.SetCoefficient(x[i * data["ships"] + j], data["capacity_matrix"][i][j])
    
    for i in range(data["ships"]):
        constraint = solver.RowConstraint(0, data["total_ships"][i], f"ship_total_{i}")
        for j in range(data["course"]):
            constraint.SetCoefficient(x[j * data["ships"] + i], 1)
            
    
    objective = solver.Objective()
    
    # print([data["total_ships"][i] * data["vacant_cost"][i] for i in range(data["ships"])])
    
    objective.SetOffset(sum([data["total_ships"][i] * data["vacant_cost"][i] for i in range(data["ships"])]))
    
    for i in range(data["course"]):
        
        for j in range(data["ships"]):
            objective.SetCoefficient(x[i * data["ships"] + j], data["od_coefficient_matrix"][i][j] - data["vacant_cost"][j])
    
    objective.SetMinimization()
    
    status = solver.Solve()
    # 输出求解结果
    if status == pywraplp.Solver.OPTIMAL:
        print("Objective value =", solver.Objective().Value())
        
        for i in range(data["course"]):
            for j in range(data["ships"]):
                print(f"第{i + 1}条航线，用第{j + 1}种船的数量是： ",   " = " , x[i * data["ships"] + j].solution_value())
        # for j in range(data["num_vars"]):
        #     print(x[j].name(), " = ", x[j].solution_value())
        print()
        print("Problem solved in %f milliseconds" % solver.wall_time())
        print("Problem solved in %d iterations" % solver.iterations())
        print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
    else:
        print("The problem does not have an optimal solution.")
    
    
if __name__ == "__main__":
    solve_transportation_model()
    


Number of variables = 30
Objective value = 749983.95
第1条航线，用第1种船的数量是：   =  0.0
第1条航线，用第2种船的数量是：   =  2.0
第1条航线，用第3种船的数量是：   =  2.0
第1条航线，用第4种船的数量是：   =  -0.0
第1条航线，用第5种船的数量是：   =  -0.0
第2条航线，用第1种船的数量是：   =  0.0
第2条航线，用第2种船的数量是：   =  0.0
第2条航线，用第3种船的数量是：   =  5.0
第2条航线，用第4种船的数量是：   =  -0.0
第2条航线，用第5种船的数量是：   =  -0.0
第3条航线，用第1种船的数量是：   =  0.0
第3条航线，用第2种船的数量是：   =  2.0
第3条航线，用第3种船的数量是：   =  0.0
第3条航线，用第4种船的数量是：   =  0.0
第3条航线，用第5种船的数量是：   =  2.0
第4条航线，用第1种船的数量是：   =  -0.0
第4条航线，用第2种船的数量是：   =  -0.0
第4条航线，用第3种船的数量是：   =  -0.0
第4条航线，用第4种船的数量是：   =  11.0
第4条航线，用第5种船的数量是：   =  -0.0
第5条航线，用第1种船的数量是：   =  -0.0
第5条航线，用第2种船的数量是：   =  0.0
第5条航线，用第3种船的数量是：   =  1.0
第5条航线，用第4种船的数量是：   =  1.0
第5条航线，用第5种船的数量是：   =  0.0
第6条航线，用第1种船的数量是：   =  0.0
第6条航线，用第2种船的数量是：   =  1.0
第6条航线，用第3种船的数量是：   =  -0.0
第6条航线，用第4种船的数量是：   =  0.0
第6条航线，用第5种船的数量是：   =  8.0

Problem solved in 21.000000 milliseconds
Problem solved in 401 iterations
Problem solved in 2 branch-and-bound nodes


In [3]:

def main():
    data = create_data_model()
    # 用SCIP方法创建 MIP 求解器
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return

    infinity = solver.infinity()
    x = {}
    for j in range(data["num_vars"]):
        x[j] = solver.IntVar(0, infinity, "x[%i]" % j)
    print("Number of variables =", solver.NumVariables())

    for i in range(data["num_constraints"]):
        # 可以灵活地用遍历列表的方式给参数赋值，这里RowConstraint对应前面说的MakeRowConstraint方法
        constraint = solver.RowConstraint(0, data["bounds"][i], "")
        for j in range(data["num_vars"]):
            constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j])
    print("Number of constraints =", solver.NumConstraints())
    # 也可以使用如下方法创建约束
    # for i in range(data['num_constraints']):
    #  constraint_expr = \
    # [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
    #  solver.Add(sum(constraint_expr) <= data['bounds'][i])

    objective = solver.Objective()
    for j in range(data["num_vars"]):
        objective.SetCoefficient(x[j], data["obj_coeffs"][j])
    objective.SetMaximization()
    # 也可以使用如下方法设置目标函数
    # obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
    # solver.Maximize(solver.Sum(obj_expr))

    status = solver.Solve()
    # 输出求解结果
    if status == pywraplp.Solver.OPTIMAL:
        print("Objective value =", solver.Objective().Value())
        for j in range(data["num_vars"]):
            print(x[j].name(), " = ", x[j].solution_value())
        print()
        print("Problem solved in %f milliseconds" % solver.wall_time())
        print("Problem solved in %d iterations" % solver.iterations())
        print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
    else:
        print("The problem does not have an optimal solution.")

def create_data_model():
    """用dict来存储模型的数据"""
    data = {}
    data["constraint_coeffs"] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data["bounds"] = [250, 285, 211, 315]
    data["obj_coeffs"] = [7, 8, 2, 9, 6]
    data["num_vars"] = 5
    data["num_constraints"] = 4
    return data

if __name__ == "__main__":
    main()


Number of variables = 5
Number of constraints = 4
Objective value = 260.0000000000003
x[0]  =  8.0
x[1]  =  21.0
x[2]  =  0.0
x[3]  =  2.0
x[4]  =  3.0

Problem solved in 16.000000 milliseconds
Problem solved in 66 iterations
Problem solved in 7 branch-and-bound nodes


# Bin Packing Problem (BPP)

> Solved via SCIP.

In [4]:
##@file bpp.py 
# @brief use SCIP for solving the bin packing problem.
"""
The instance of the bin packing problem is represented by the two
lists of n items of sizes and quantity s=(s_i).
The bin size is B.

We use Martello and Toth (1990) formulation, and suggest
extensions with tie-breaking and SOS constraints.

Copyright (c) by Joao Pedro PEDROSO and Mikio KUBO, 2012
"""

from pyscipopt import Model, quicksum


def FFD(s, B):
    """First Fit Decreasing heuristics for the Bin Packing Problem.
    Parameters:
        - s: list with item widths
        - B: bin capacity
    Returns a list of lists with bin compositions.
    """
    remain = [B]  # keep list of empty space per bin
    sol = [[]]  # a list ot items (i.e., sizes) on each used bin
    for item in sorted(s, reverse=True):
        for (j, free) in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:  # does not fit in any bin
            sol.append([item])
            remain.append(B - item)
    return sol


def bpp(s, B):
    """bpp: Martello and Toth's model to solve the bin packing problem.
    Parameters:
        - s: list with item widths
        - B: bin capacity
    Returns a model, ready to be solved.
    """
    n = len(s)
    U = len(FFD(s, B))  # upper bound of the number of bins
    model = Model("bpp")
    # setParam("MIPFocus",1)
    x, y = {}, {}
    for i in range(n):
        for j in range(U):
            x[i, j] = model.addVar(vtype="B", name="x(%s,%s)" % (i, j))
    for j in range(U):
        y[j] = model.addVar(vtype="B", name="y(%s)" % j)

    # assignment constraints
    for i in range(n):
        model.addCons(quicksum(x[i, j] for j in range(U)) == 1, "Assign(%s)" % i)

    # bin capacity constraints
    for j in range(U):
        model.addCons(quicksum(s[i] * x[i, j] for i in range(n)) <= B * y[j], "Capac(%s)" % j)

    # tighten assignment constraints
    for j in range(U):
        for i in range(n):
            model.addCons(x[i, j] <= y[j], "Strong(%s,%s)" % (i, j))

    # tie breaking constraints
    for j in range(U - 1):
        model.addCons(y[j] >= y[j + 1], "TieBrk(%s)" % j)

    # SOS constraints
    # SOS约束包括SOS1约束和SOS2约束
    # SOS1约束要求一组变量中至多有1个变量的取值不为零；（就是 sumation <= 1）
    # SOS2约束要求一组变量中至多有2个变量的取值不为零、且非零取值的变量必须是相邻变量。
    for i in range(n):
        model.addConsSOS1([x[i, j] for j in range(U)])

    model.setObjective(quicksum(y[j] for j in range(U)), "minimize")
    model.data = x, y

    return model


def solveBinPacking(s, B):
    """solveBinPacking: use an IP model to solve the in Packing Problem.

    Parameters:
        - s: list with item widths
        - B: bin capacity

    Returns a solution: list of lists, each of which with the items in a roll.
    """
    n = len(s)
    U = len(FFD(s, B))  # upper bound of the number of bins
    model = bpp(s, B)
    x, y = model.data

    model.optimize()

    bins = [[] for i in range(U)]
    for (i, j) in x:
        if model.getVal(x[i, j]) > .5:
            bins[j].append(s[i])
    for i in range(bins.count([])):
        bins.remove([])
    for b in bins:
        b.sort()
    bins.sort()
    return bins


import random


def DiscreteUniform(n=20, LB=1, UB=99, B=100):
    """DiscreteUniform: create random, uniform instance for the bin packing problem."""
    B = 100
    s = [0] * n
    for i in range(n):
        s[i] = random.randint(LB, UB)
    return s, B


if __name__ == "__main__":
    random.seed(256)
    s, B = DiscreteUniform()
    print("items:", s)
    print("bin size:", B)

    ffd = FFD(s, B)
    print("\n\n\n FFD heuristic:")
    print("Solution:")
    print(ffd)
    print(len(ffd), "bins")

    print("\n\n\n IP formulation:")
    bins = solveBinPacking(s, B)
    print("Solution:")
    print(bins)
    print(len(bins), "bins")

items: [63, 40, 56, 47, 49, 6, 59, 99, 56, 32, 66, 23, 37, 86, 41, 72, 42, 43, 78, 11]
bin size: 100



 FFD heuristic:
Solution:
[[99], [86, 11], [78, 6], [72, 23], [66, 32], [63, 37], [59, 41], [56, 43], [56, 42], [49, 47], [40]]
11 bins



 IP formulation:
presolving:
(round 1, medium)     0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 20 upgd conss, 0 impls, 371 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 281 upgd conss, 0 impls, 371 clqs
(round 3, fast)       0 del vars, 20 del conss, 0 add conss, 0 chg bounds, 20 chg sides, 33 chg coeffs, 281 upgd conss, 0 impls, 371 clqs
   (0.0s) probing cycle finished: starting next cycle
(round 4, exhaustive) 2 del vars, 20 del conss, 0 add conss, 0 chg bounds, 20 chg sides, 33 chg coeffs, 281 upgd conss, 0 impls, 519 clqs
(round 5, fast)       2 del vars, 62 del conss, 0 add conss, 0 chg bounds, 20 chg sides, 33 chg coeffs, 281 upgd conss, 0 impls, 

/var/folders/wp/zfkgqzvj2lq6fk2gy0rbmq6m0000gn/T/ipykernel_88770/288140438.py:47: UserWarning: linked SCIP 9.02 is not recommended for this version of PySCIPOpt - use version 9.2.1
  model = Model("bpp")
